In [1]:
### GIS-based bikeshare optimization program ###
################################################################################################
''' Copyright (C) 2023 by Hannah DeBruin
Released under the GNU General Public License, version 2.
-------------------------------------------------------
The code is entirely and originally written by Hannah DeBruin
Contact: hannah.debruin@gmail.com
-------------------------------------------------------
This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.
You should have received a copy of the GNU General Public License
along with this program. If not, see <http://www.gnu.org/licenses/>. 

The .csv files referenced in this text are outputs of the ArcGIS Pro
"Bikeshare Optimization Data Tool" which can be found at the link:
<https://github.umn.edu/TransitLab/bikeshare-station-optimization> '''
################################################################################################
################################################################################################
################################################################################################

from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

from __future__ import print_function
import sys
import cplex
from cplex.exceptions import CplexError

Current Time = 11:31:22


In [2]:
import docplex.mp

In [3]:
import numpy as np

In [4]:
import scipy

In [5]:
from docplex.mp.model import Model

In [6]:
m = Model('bike_station')

In [7]:
#specifying total budgeted number of stations
N_tot = 40
#specifying minimum number stations meeting each goal
n_reg = 10
n_rec = 6
n_eq = 10
n_trans = 9
n_ppp = 5
N_CL = 400

In [8]:
#determining total number of grid spaces
import csv

f = open('regular_use_CL.csv', newline='')
n = len(f.readlines())
    
fR = range(0,n-1)

f.close()

In [9]:
# variables associated with each grid space
fw_reg = m.continuous_var_list(fR, lb=None, ub=None, name="fW_reg", key_format=None)
fb_reg = m.binary_var_list(fR, lb=None, ub=None, name="fb_reg", key_format=None)
fb_reg = [0]*(n-1)
fw_rec = m.continuous_var_list(fR, lb=None, ub=None, name="fW_rec", key_format=None)
fb_rec = m.binary_var_list(fR, lb=None, ub=None, name="fb_rec", key_format=None)
fb_rec = [0]*(n-1)
fw_eq = m.continuous_var_list(fR, lb=None, ub=None, name="fW_eq", key_format=None)
fb_eq = m.binary_var_list(fR, lb=None, ub=None, name="fb_eq", key_format=None)
fb_eq = [0]*(n-1)
fw_trans = m.continuous_var_list(fR, lb=None, ub=None, name="fW_trans", key_format=None)
fb_trans = m.binary_var_list(fR, lb=None, ub=None, name="fb_trans", key_format=None)
fb_trans = [0]*(n-1)
fw_ppp = m.continuous_var_list(fR, lb=None, ub=None, name="fW_ppp", key_format=None)
fb_ppp = m.binary_var_list(fR, lb=None, ub=None, name="fb_ppp", key_format=None)
fb_ppp = [0]*(n-1)
ffid = m.integer_var_list(fR, lb=None, ub=None, name="fFid", key_format=None)
flat = m.continuous_var_list(fR, lb=None, ub=None, name="flat", key_format=None)
flong = m.continuous_var_list(fR, lb=None, ub=None, name="flong", key_format=None)

#reading in point data for regular use
with open('regular_use_CL.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)
    for i in fR:
        ffid[i] = int(data[i+1][1])
        fw_reg[i] = float(data[i+1][2])
        flat[i] = float(data[i+1][3])
        flong[i] = float(data[i+1][4])
        if i < N_CL: #if in top-ranking locations
            fb_reg[i] = 1

f.close()

#adding in point data for recreational use
with open('recreational_use_CL.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)
    for i in fR:
        for j in fR:
            if ffid[i] == int(data[j+1][1]):
                fw_rec[i] = float(data[j+1][2])
                if j < N_CL: #if in top-ranking locations
                    fb_rec[i] = 1
f.close()

#adding in point data for equity
with open('equity_CL.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)
    for i in fR:
        for j in fR:
            if ffid[i] == int(data[j+1][1]):
                fw_eq[i] = float(data[j+1][2])
                if j< N_CL: #if in top-ranking locations
                    fb_eq[i] = 1
f.close()


#adding in point data for transportation network
with open('trans_net_CL.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)
    for i in fR:
        for j in fR:
            if ffid[i] == int(data[j+1][1]):
                fw_trans[i] = float(data[j+1][2])
                if j< N_CL: #if in top-ranking locations
                    fb_trans[i] = 1
f.close()

#adding in point data for partnership opportunities
with open('partnerships_CL.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)
    for i in fR:
        for j in fR:
            if ffid[i] == int(data[j+1][1]):
                fw_ppp[i] = float(data[j+1][2])
                if j< N_CL: #if in top-ranking locations
                    fb_ppp[i] = 1
f.close()

#creating subset of just top-ranking locations
subset_size = 0
for i in fR:
    if (fb_reg[i] + fb_rec[i] + fb_eq[i] + fb_trans[i] + fb_ppp[i]) >= 1:
        subset_size = subset_size + 1


R = range(0,subset_size)

print("Candidate Locations Subset Size:",subset_size)

# variables associated with each subset candidate location
w_reg = m.continuous_var_list(R, lb=None, ub=None, name="W_reg", key_format=None)
b_reg = m.binary_var_list(R, lb=None, ub=None, name="b_reg", key_format=None)
w_rec = m.continuous_var_list(R, lb=None, ub=None, name="W_rec", key_format=None)
b_rec = m.binary_var_list(R, lb=None, ub=None, name="b_rec", key_format=None)
w_eq = m.continuous_var_list(R, lb=None, ub=None, name="W_eq", key_format=None)
b_eq = m.binary_var_list(R, lb=None, ub=None, name="b_eq", key_format=None)
w_trans = m.continuous_var_list(R, lb=None, ub=None, name="W_trans", key_format=None)
b_trans = m.binary_var_list(R, lb=None, ub=None, name="b_trans", key_format=None)
w_ppp = m.continuous_var_list(R, lb=None, ub=None, name="W_ppp", key_format=None)
b_ppp = m.binary_var_list(R, lb=None, ub=None, name="b_ppp", key_format=None)
fid = m.integer_var_list(R, lb=None, ub=None, name="Fid", key_format=None)
w = m.continuous_var_list(R, lb=None, ub=None, name="W", key_format=None)
lat = m.continuous_var_list(fR, lb=None, ub=None, name="lat", key_format=None)
long = m.continuous_var_list(fR, lb=None, ub=None, name="long", key_format=None)

#assigning overall weight, all data to candidate location subset
j = 0
for i in fR:
    if (fb_reg[i] + fb_rec[i] + fb_eq[i] + fb_trans[i] + fb_ppp[i]) >= 1:
        fid[j] = ffid[i]
        w_reg[j] = fw_reg[i]
        b_reg[j] = fb_reg[i]
        w_rec[j] = fw_rec[i]
        b_rec[j] = fb_rec[i]
        w_eq[j] = fw_eq[i]
        b_eq[j] = fb_eq[i]
        w_trans[j] = fw_trans[i]
        b_trans[j] = fb_trans[i]
        w_ppp[j] = fw_ppp[i]
        b_ppp[j] = fb_ppp[i]
        lat[j] = flat[i]
        long[j] = flong[i]
        w[j] = ((n_reg/N_tot)*fw_reg[i]) + ((n_rec/N_tot)*fw_rec[i]) + ((n_eq/N_tot)*fw_eq[i]) + ((n_trans/N_tot)*fw_trans[i])+ ((n_ppp/N_tot)*fw_ppp[i])
        j = j+1

Candidate Locations Subset Size: 1283


In [10]:
#defining variable for yes/no in each grid cell
x = m.binary_var_list(R, lb=None, ub=None, name="X", key_format=None)

In [11]:
# max number of stations constraint
m.add_constraint(m.sum(x) <= N_tot)
# min number of stations for each goal constraint
m.add_constraint(m.dot(x,b_reg) >= n_reg)
m.add_constraint(m.dot(x,b_rec) >= n_rec)
m.add_constraint(m.dot(x,b_eq) >= n_eq)
m.add_constraint(m.dot(x,b_trans) >= n_trans)
m.add_constraint(m.dot(x,b_ppp) >= n_ppp)

docplex.mp.LinearConstraint[](X_0+X_1+X_2+X_3+X_5+X_6+X_9+X_29+X_33+X_38+X_74+X_80+X_83+X_87+X_90+X_91+X_92+X_99+X_126+X_134+X_160+X_164+X_167+X_168+X_171+X_172+X_174+X_175+X_177+X_181+X_185+X_192+X_194+X_197+X_216+X_217+X_219+X_225+X_232+X_252+X_257+X_265+X_268+X_272+X_281+X_289+X_291+X_298+X_301+X_309+X_312+X_313+X_314+X_319+X_325+X_331+X_334+X_342+X_346+X_348+X_351+X_353+X_356+X_360+X_361+X_371+X_377+X_382+X_389+X_396+X_403+X_404+X_405+X_406+X_408+X_416+X_417+X_420+X_423+X_424+X_425+X_426+X_427+X_428+X_430+X_432+X_433+X_434+X_435+X_436+X_437+X_438+X_439+X_440+X_442+X_443+X_445+X_447+X_448+X_453+X_456+X_459+X_460+X_465+X_467+X_469+X_470+X_476+X_478+X_480+X_482+X_483+X_484+X_486+X_487+X_490+X_491+X_492+X_493+X_494+X_495+X_496+X_497+X_503+X_504+X_508+X_512+X_517+X_519+X_520+X_523+X_524+X_525+X_527+X_528+X_531+X_533+X_534+X_538+X_542+X_544+X_547+X_550+X_551+X_552+X_553+X_554+X_555+X_561+X_562+X_564+X_566+X_570+X_573+X_579+X_585+X_588+X_590+X_599+X_601+X_605+X_607+X_618+X_620+X_621+X_622

In [12]:
#reading in zone data

#regular use
import csv
zf_reg = open('regular_use_ZT.csv', newline='')
zn_reg = len(zf_reg.readlines())
zR_reg = range(0,zn_reg-1)
zf_reg.close()

s_1_reg = m.integer_var_list(zR_reg, lb=None, ub=None, name="S_1_reg", key_format=None)
s_2_reg = m.integer_var_list(zR_reg, lb=None, ub=None, name="S_2_reg", key_format=None)
zfid_reg = m.integer_var_list(zR_reg, lb=None, ub=None, name="zFid_reg", key_format=None)

with open('regular_use_ZT.csv', 'r') as zf_reg:
    reader = csv.reader(zf_reg)
    data = list(reader)
    for i in zR_reg:
        zfid_reg[i] = int(data[i+1][2])
        s_1_reg[i] = int(data[i+1][6])
        s_2_reg[i] = int(data[i+1][9])
        
#determine number of zones
nzones_reg = m.max(m.max(s_1_reg),m.max(s_2_reg))
Z_reg = range(0,nzones_reg)   
print(Z_reg)

#recreational use
import csv
zf_rec = open('recreational_use_ZT.csv', newline='')
zn_rec = len(zf_rec.readlines())
zR_rec = range(0,zn_rec-1)
zf_rec.close()

s_1_rec = m.integer_var_list(zR_rec, lb=None, ub=None, name="S_1_rec", key_format=None)
s_2_rec = m.integer_var_list(zR_rec, lb=None, ub=None, name="S_2_rec", key_format=None)
zfid_rec = m.integer_var_list(zR_rec, lb=None, ub=None, name="zFid_rec", key_format=None)

with open('recreational_use_ZT.csv', 'r') as zf_rec:
    reader = csv.reader(zf_rec)
    data = list(reader)
    for i in zR_rec:
        zfid_rec[i] = int(data[i+1][2])
        s_1_rec[i] = int(data[i+1][6])
        s_2_rec[i] = int(data[i+1][9])
        
#determine number of zones
nzones_rec = m.max(m.max(s_1_rec),m.max(s_2_rec))
Z_rec = range(0,nzones_rec)  
print(Z_rec)

#equity
import csv
zf_eq = open('equity_ZT.csv', newline='')
zn_eq = len(zf_eq.readlines())
zR_eq = range(0,zn_eq-1)
zf_eq.close()

s_1_eq = m.integer_var_list(zR_eq, lb=None, ub=None, name="S_1_eq", key_format=None)
s_2_eq = m.integer_var_list(zR_eq, lb=None, ub=None, name="S_2_eq", key_format=None)
zfid_eq = m.integer_var_list(zR_eq, lb=None, ub=None, name="zFid_eq", key_format=None)

with open('equity_ZT.csv', 'r') as zf_eq:
    reader = csv.reader(zf_eq)
    data = list(reader)
    for i in zR_eq:
        zfid_eq[i] = int(data[i+1][2])
        s_1_eq[i] = int(data[i+1][6])
        s_2_eq[i] = int(data[i+1][9])
        
#determine number of zones
nzones_eq = m.max(m.max(s_1_eq),m.max(s_2_eq))
Z_eq = range(0,nzones_eq)   
print(Z_eq)

#transportation network
import csv
zf_trans = open('trans_net_ZT.csv', newline='')
zn_trans = len(zf_trans.readlines())
zR_trans = range(0,zn_trans-1)
zf_trans.close()

s_1_trans = m.integer_var_list(zR_trans, lb=None, ub=None, name="S_1_trans", key_format=None)
s_2_trans = m.integer_var_list(zR_trans, lb=None, ub=None, name="S_2_trans", key_format=None)
zfid_trans = m.integer_var_list(zR_trans, lb=None, ub=None, name="zFid_trans", key_format=None)

with open('trans_net_ZT.csv', 'r') as zf_trans:
    reader = csv.reader(zf_trans)
    data = list(reader)
    for i in zR_trans:
        zfid_trans[i] = int(data[i+1][2])
        s_1_trans[i] = int(data[i+1][6])
        s_2_trans[i] = int(data[i+1][9])
        
#determine number of zones
nzones_trans = m.max(m.max(s_1_trans),m.max(s_2_trans))
Z_trans = range(0,nzones_trans)   
print(Z_trans)


#partnership opportunities
import csv
zf_ppp = open('partnerships_ZT.csv', newline='')
zn_ppp = len(zf_ppp.readlines())
zR_ppp = range(0,zn_ppp-1)
zf_ppp.close()

s_1_ppp = m.integer_var_list(zR_ppp, lb=None, ub=None, name="S_1_ppp", key_format=None)
s_2_ppp = m.integer_var_list(zR_ppp, lb=None, ub=None, name="S_2_ppp", key_format=None)
zfid_ppp = m.integer_var_list(zR_ppp, lb=None, ub=None, name="zFid_ppp", key_format=None)

with open('partnerships_ZT.csv', 'r') as zf_ppp:
    reader = csv.reader(zf_ppp)
    data = list(reader)
    for i in zR_ppp:
        zfid_ppp[i] = int(data[i+1][2])
        s_1_ppp[i] = int(data[i+1][6])
        s_2_ppp[i] = int(data[i+1][9])
        
#determine number of zones
nzones_ppp = m.max(m.max(s_1_ppp),m.max(s_2_ppp))
Z_ppp = range(0,nzones_ppp)   
print(Z_ppp)

range(0, 10)
range(0, 6)
range(0, 10)
range(0, 9)
range(0, 5)


In [13]:
# cover each set constraint

#regular use
count_reg = m.integer_var_list(Z_reg, lb=None, ub=None, name="count_reg", key_format=None)
count_reg = [0]*nzones_reg

for i in zR_reg:
    for j in R:
        for k in Z_reg:
            if zfid_reg[i] == fid[j]:
                if s_1_reg[i] == (k+1) or s_2_reg[i] == (k+1):
                    count_reg[k] = count_reg[k] + x[j]
                    
for i in Z_reg:
    m.add_constraint(count_reg[i] >= 1)
    
#recreational use
count_rec = m.integer_var_list(Z_rec, lb=None, ub=None, name="count_rec", key_format=None)
count_rec = [0]*nzones_rec

for i in zR_rec:
    for j in R:
        for k in Z_rec:
            if zfid_rec[i] == fid[j]:
                if s_1_rec[i] == (k+1) or s_2_rec[i] == (k+1):
                    count_rec[k] = count_rec[k] + x[j]
                    
for i in Z_rec:
    m.add_constraint(count_rec[i] >= 1) 
    
#equity
count_eq = m.integer_var_list(Z_eq, lb=None, ub=None, name="count_eq", key_format=None)
count_eq = [0]*nzones_eq

for i in zR_eq:
    for j in R:
        for k in Z_eq:
            if zfid_eq[i] == fid[j]:
                if s_1_eq[i] == (k+1) or s_2_eq[i] == (k+1):
                    count_eq[k] = count_eq[k] + x[j]
                    
for i in Z_eq:
    m.add_constraint(count_eq[i] >= 1) 
    
#transportation network
count_trans = m.integer_var_list(Z_trans, lb=None, ub=None, name="count_trans", key_format=None)
count_trans = [0]*nzones_trans

for i in zR_trans:
    for j in R:
        for k in Z_trans:
            if zfid_trans[i] == fid[j]:
                if s_1_trans[i] == (k+1) or s_2_trans[i] == (k+1):
                    count_trans[k] = count_trans[k] + x[j]
                    
for i in Z_trans:
    m.add_constraint(count_trans[i] >= 1) 
    
#partnership opportunities
count_ppp = m.integer_var_list(Z_ppp, lb=None, ub=None, name="count_ppp", key_format=None)
count_ppp = [0]*nzones_ppp

for i in zR_ppp:
    for j in R:
        for k in Z_ppp:
            if zfid_ppp[i] == fid[j]:
                if s_1_ppp[i] == (k+1) or s_2_ppp[i] == (k+1):
                    count_ppp[k] = count_ppp[k] + x[j]
                    
for i in Z_ppp:
    m.add_constraint(count_ppp[i] >= 1) 

In [14]:
#read distance file
df = open('DistanceTable.csv', 'r')
dn = len(df.readlines())

dR = range(0,dn-1)

df.close()

o = m.integer_var_list(dR, lb=None, ub=None, name="o", key_format=None)
d = m.integer_var_list(dR, lb=None, ub=None, name="d", key_format=None)

with open('DistanceTable.csv', newline='') as df:
    reader = csv.reader(df)
    data = list(reader)
    for i in dR:
        o[i] = int(data[i+1][1])
        d[i] = int(data[i+1][2])
        
for i in dR:
    for j in R:
        if o[i] == fid[j]:
            for k in R:
                if d[i] == fid[k]:
                    m.add_constraint(x[j]+x[k] <= 1)
        

In [18]:
obj = m.dot(x,w)
m.maximize(obj)
solution = m.solve()
assert solution
print(solution)

solution for: bike_station
objective: 2172.9
X_1=1
X_2=1
X_18=1
X_24=1
X_36=1
X_37=1
X_54=1
X_57=1
X_67=1
X_70=1
X_80=1
X_126=1
X_175=1
X_184=1
X_202=1
X_334=1
X_348=1
X_358=1
X_372=1
X_405=1
X_410=1
X_420=1
X_421=1
X_440=1
X_443=1
X_454=1
X_466=1
X_478=1
X_486=1
X_504=1
X_539=1
X_583=1
X_589=1
X_593=1
X_638=1
X_687=1
X_693=1
X_722=1
X_780=1
X_895=1



NameError: name 'parameters' is not defined

In [16]:
#printing solutions
with open('solution.csv','w',newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["fid","x","b_reg","b_rec","b_eq","b_trans","b_ppp","lat","long"])
    for i in R:
        if solution[x[i]] == 1:
            writer.writerow([fid[i],solution[x[i]],b_reg[i],b_rec[i],b_eq[i],b_trans[i],b_ppp[i],lat[i],long[i]])

In [17]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 11:40:22
